## Optimize the placement of 5G towers using Graph Theory

In this project, I've developed an application to optimize the placement of 5G towers in a simulated environment. The app, built with Dash, displays a grid representing different locations, each with a random population. The main goal is to place towers efficiently to cover as many people as possible.

**Key points:**
- The grid size is set to 10x10 with 30 populated locations, and each tower covers a radius of 2 units.
- Locations are sorted by population to prioritize coverage.
- Users can interact with the app to optimize tower placement or refresh the locations.
- The app visually represents the optimization process, showing where towers are placed in relation to the population centers.
- This tool can be incredibly useful for telecommunications planning, providing a visual and interactive way to strategize the deployment of network infrastructure.

### Import the libraries

In [1]:
import dash
import dash_cytoscape as cyto
import dash_html_components as html
import dash_bootstrap_components as dbc
import dash_core_components as dcc
from dash.dependencies import Input, Output
import networkx as nx
import random

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """


### Defining the Grid size, number of locations and Tower radius
The code defines constants for a simulation to optimize 5G tower placement: a 10x10 grid with 30 populated locations and each tower having a coverage radius of 2 units

In [2]:
# Constants
GRID_SIZE = 10
NUM_LOCATIONS = 30  # Number of populated locations
TOWER_RADIUS = 2  # Maximum coverage radius of a tower

### Generating and Sorting Populated Locations for Optimization Analysis
The code creates a list of randomly populated locations with varying populations, each assigned a unique identifier and coordinates. It then sorts these locations in descending order based on their population, prioritizing more densely populated areas.

In [3]:
# Create random populated locations with random population
locations = [{"id": f"L{x}_{y}", "position": {"x": x * 100, "y": y * 100}, "population": random.randint(1, 100)} 
             for x, y in random.sample([(i, j) for i in range(GRID_SIZE) for j in range(GRID_SIZE)], NUM_LOCATIONS)]

locations.sort(key=lambda x: x["population"], reverse=True)  # Sort locations by population

### Interactive 5G Tower Optimization Dashboard with Dash and Cytoscape
This code sets up an interactive web application using Dash to optimize the placement of 5G towers. It defines a visual style for network nodes and towers, and arranges them in a grid layout. The app allows users to either refresh the grid with new random locations or optimize tower placement to cover the most populated areas. The optimization logic is triggered by buttons in the interface, and the results, including the number of placed towers and their locations, are dynamically displayed in the app.

In [4]:
# Define stylesheet
stylesheet = [
    {
        'selector': 'node',
        'style': {
            'content': 'data(id)',
            'color': 'black',
            'background-color': 'blue',
            'width': 'mapData(population, 0, 100, 10, 50)',
            'height': 'mapData(population, 0, 100, 10, 50)'
        }
    },
    {
        'selector': '.tower',
        'style': {
            'background-color': 'red',
            'width': '50px',
            'height': '50px'
        }
    }
]
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row(
        dbc.Col(html.H2("5G Tower Optimization"), width=12)
    ),
    dbc.Row(
        dbc.Col(cyto.Cytoscape(
            id='cytoscape-elements',
            layout={'name': 'preset'},
            style={'width': '100%', 'height': '800px'},
            elements=[{"data": {"id": loc["id"], "population": loc["population"]}, 
                       "position": loc["position"]} for loc in locations],
            stylesheet=stylesheet
        ), width=12)
    ),
    dbc.Row([
        dbc.Col(dbc.Button("Optimize Towers", id="optimize-button", color="primary"), width=6),
        dbc.Col(dbc.Button("Refresh", id="refresh-button", color="secondary"), width=6)
    ]),
    dbc.Row(
        dbc.Col(html.Div(id='output-data'), width=12)
    )
], fluid=True)


@app.callback(
    [Output('cytoscape-elements', 'elements'),
     Output('output-data', 'children')],
    [Input('optimize-button', 'n_clicks'),
     Input('refresh-button', 'n_clicks')],
    prevent_initial_call=True
)
def update_graph(optimize_clicks, refresh_clicks):
    ctx = dash.callback_context
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    # If refresh button was clicked
    if button_id == 'refresh-button':
        global locations
        locations = [{"id": f"L{x}_{y}", "position": {"x": x * 100, "y": y * 100}, "population": random.randint(1, 100)} 
                     for x, y in random.sample([(i, j) for i in range(GRID_SIZE) for j in range(GRID_SIZE)], NUM_LOCATIONS)]
        locations.sort(key=lambda x: x["population"], reverse=True)
        return [{"data": {"id": loc["id"], "population": loc["population"]}, "position": loc["position"]} for loc in locations], dash.no_update
    
    # If optimize button was clicked
    elif button_id == 'optimize-button':
        # (You'll need to replace this comment with the logic you have for the optimization)
        # For instance:
        uncovered_locations = locations.copy()
        towers = []
        while uncovered_locations:
            tower_position = uncovered_locations[0]["position"]
            towers.append({
                "data": {"id": f"Tower_{len(towers)}"},
                "position": tower_position,
                "classes": "tower"
            })
            uncovered_locations = [loc for loc in uncovered_locations 
                                   if abs(loc["position"]["x"] - tower_position["x"]) > TOWER_RADIUS * 100 or 
                                   abs(loc["position"]["y"] - tower_position["y"]) > TOWER_RADIUS * 100]

        return [{"data": {"id": loc["id"], "population": loc["population"]}, 
                 "position": loc["position"]} for loc in locations] + towers, f"Placed {len(towers)} towers."

if __name__ == '__main__':
    app.run_server(debug=True)